In [1]:
import pyvista as pv
import numpy as np
import os, flopy
import matplotlib.pyplot as plt
name = 'Model'
workspace = '../model'
mf_exe_name = '../exe/mf6'

sim = flopy.mf6.MFSimulation.load(sim_name=name, exe_name=mf_exe_name, sim_ws=workspace)

mfmodel = sim.get_model(model_name='model')

fname = os.path.join(workspace, name + '.hds')
hdobj = flopy.utils.HeadFile(fname, precision='double')
head = hdobj.get_data()

mtop = np.ones([mfmodel.disv.nvert.array ])*mfmodel.disv.top[0]
verticesXYArray = np.dstack((mfmodel.disv.vertices.array.xv,mfmodel.disv.vertices.array.yv))[0]

flopy is installed in /home/gida2/.local/lib/python3.6/site-packages/flopy
loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package npf...
    loading package ic...
    loading package rch...
    loading package evt...
    loading package drn...
    loading package oc...
  loading ims package model...


In [2]:
modelz = np.vstack((mfmodel.disv.top.array,mfmodel.disv.botm.array))
nlay = mfmodel.disv.nlay.array

### for model geometry

In [3]:

cellTubes = {}
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    #create polydata for lay
    layerTubes = {}
    for lay in range(nlay):
        filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelz[lay,index]] for vertex in vertexIndexList])
        cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
        #cellSurf.cell_arrays['head'] = head[lay][0][index]
        cellZ = modelz[:,index]
        cellMesh = cellSurf.extrude([0, 0, cellZ[lay+1]-cellZ[lay]])
        layerTubes[str(lay)] = cellMesh  
    blocks = pv.MultiBlock(layerTubes)
    cellTubes[str(index)] = blocks.combine()
totalModelCells = pv.MultiBlock(cellTubes)
totalModelMerged = totalModelCells.combine()
totalModelMerged.save('../vtk/modelGeometry.vtk')

### for cell heads

In [ ]:

cellTubes = {}
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    #create polydata for lay
    layerTubes = {}
    for lay in range(nlay):
        if head[lay][0][index] > -1.00000000e+30:
            filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelz[lay,index]] for vertex in vertexIndexList])
            cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
            cellSurf.cell_arrays['head'] = head[lay][0][index]
            cellZ = modelz[:,index]
            cellMesh = cellSurf.extrude([0, 0, cellZ[lay+1]-cellZ[lay]])
            layerTubes[str(lay)] = cellMesh 
    blocks = pv.MultiBlock(layerTubes)
    cellTubes[str(index)] = cellSurf
    if index % 3000 == 0:
        print(index)
totalModelCells = pv.MultiBlock(cellTubes)
totalModelMerged = totalModelCells.combine()
totalModelMerged.save('../vtk/modelCellHeads.vtk')

0
3000
6000
9000
12000
15000
18000


### for water table

In [ ]:

cellTubes = {}
for index, cell in enumerate(mfmodel.disv.cell2d.array):
    vertexIndexList = [x for x in list(cell)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    #create polydata for lay
    layerTubes = {}
    
    headCell = head[lay][0][index]
    wtCell = headCell[headCell>-1.00000000e+30][0]
    for lay in range(nlay):
    filterVertexArray = np.array([list(verticesXYArray[vertex]) + [wtCell] for vertex in vertexIndexList])
    cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
    cellSurf.cell_arrays['watertable'] = wtCell
    cellTubes[str(index)] = cellSurf
totalModelCells = pv.MultiBlock(cellTubes)
totalModelMerged = totalModelCells.combine()
totalModelMerged.save('../vtk/modelWaterTable.vtk')

In [ ]:
### for drain boundary

In [ ]:

drnPkg = mfmodel.get_package('drn_0')
drainTubes = {}
for index, drnCell in enumerate(drnPkg.stress_period_data.array[0]):
    cellNumber = drnCell[0][1]
    cellLay = drnCell[0][0]
    cellGrid = mfmodel.disv.cell2d.array[cellNumber]
    vertexIndexList = [x for x in list(cellGrid)[4:] if x is not None]
    filterVertexIndexList = [len(vertexIndexList)] + list(range(len(vertexIndexList)))
    filterVertexIndexArray = np.array(filterVertexIndexList)
    filterVertexArray = np.array([list(verticesXYArray[vertex]) + [modelz[cellLay,cellNumber]] for vertex in vertexIndexList])
    cellSurf = pv.PolyData(filterVertexArray, filterVertexIndexArray)
    cellSurf.cell_arrays['elevation'] = drnCell[1]
    cellSurf.cell_arrays['cond'] = drnCell[2]
    cellZ = modelz[:,cellNumber]
    cellMesh = cellSurf.extrude([0, 0, cellZ[cellLay +1]-cellZ[cellLay]])
    drainTubes[str(index)] = cellMesh 
totalDrainsCells = pv.MultiBlock(drainTubes)
totalDrainsMerged = totalDrainsCells.combine()
totalDrainsMerged.save('../vtk/modelDrains.vtk')